In [1]:
import torch
assert torch.cuda.is_available(), "CUDA is not available!"

In [2]:
from muzero_baseline.muzero import MuZero

from hnefatafl.muzero import HnefataflGame, MuZeroConfig
import os
config = MuZeroConfig()
config.network = "resnet"
config.max_num_gpus = 1
config.train_on_gpu = True
config.reanalyse_on_gpu = False
config.selfplay_on_gpu = False
config.batch_size = 16
config.training_steps = 1000
config.num_workers = 16
config.max_moves = 250
config.temperature_threshold = None
config.support_size = 10
config.num_unroll_steps = 20
config.num_simulations = 50

mz = MuZero(HnefataflGame, config)

# load the model if we have a checkpoint to load (model.checkpoint in config.results_path)
run_path = "results/muzero/2023-02-24_00-14-51"
checkpoint_path = os.path.join(run_path, "model.checkpoint")
replay_buffer_path = os.path.join(run_path, "replay_buffer.pkl")

if os.path.exists(checkpoint_path):
    if os.path.exists(replay_buffer_path):
        mz.load_model(checkpoint_path, replay_buffer_path)
    else:
        mz.load_model(checkpoint_path)
if os.path.exists(replay_buffer_path):
    mz.load_model(replay_buffer_path=replay_buffer_path)


2023-02-24 00:14:52,512	INFO worker.py:1538 -- Started a local Ray instance.



Initializing replay buffer with results/muzero/2023-02-23_20-10-20/replay_buffer.pkl


In [3]:
%load_ext tensorboard
%tensorboard --logdir ./results/muzero --bind_all

In [4]:
mz.train(per_step_progress=True)

Timesteps:   0%|          | 0/16000 [00:00<?, ?it/s]


Training...
Run tensorboard --logdir ./results and go to http://localhost:6006/ to see in real time the training performance.



Timesteps:   0%|          | 13/16000 [00:20<6:50:58,  1.54s/it]

(ReplayBuffer pid=710170) Replay buffer initialized with 14307 samples (90 games).
(ReplayBuffer pid=710170) 


Timesteps:   9%|▉         | 1501/16000 [19:22<3:07:13,  1.29it/s]



Shutting down workers...


Persisting replay buffer games to disk...
